In [36]:
from triplet_models.gpt import triplet_gpt
input_text = "Subscribe Sign In Uber’s Financials: An Inside Look Uber Technologies Inc. isn't obligated to publicly disclose its financial records as a privately held company. The Wall Street Journal reviewed recent quarterly statements that show how fast the most valuable U.S. private technology company is growing — and losing money."

result_dict = triplet_gpt(input_text)
print(result_dict)
print(result_dict["subject"])
print(result_dict["subject_type"])
print(result_dict["relationship"])
print(result_dict["object"])
print(result_dict["object_type"])


{'subject': ['Uber Technologies Inc.', 'Uber Technologies Inc.'], 'subject_type': ['COMP', 'COMP'], 'relationship': ['Operate_In', 'Operate_In'], 'object': ['Technology', 'United States'], 'object_type': ['SECTOR', 'GPE']}
['Uber Technologies Inc.', 'Uber Technologies Inc.']
['COMP', 'COMP']
['Operate_In', 'Operate_In']
['Technology', 'United States']
['SECTOR', 'GPE']


In [1]:
from triplet_models.triplex import triplet_triplex
input_text = "Subscribe Sign In Uber’s Financials: An Inside Look Uber Technologies Inc. isn't obligated to publicly disclose its financial records as a privately held company. The Wall Street Journal reviewed recent quarterly statements that show how fast the most valuable U.S. private technology company is growing — and losing money."

result = triplet_triplex(input_text)
print(result)

c:\Users\KTH\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attention` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.
Loading checkpoint shards: 100%|██████████| 2/2 [01:24<00:00, 42.49s/it]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
You are not running the flash-attention implementation, expect numerical difference

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacty of 8.00 GiB of which 0 bytes is free. Of the allocated memory 14.59 GiB is allocated by PyTorch, and 40.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [1]:
from triplet_models.llama import triplet_llama
import json

input_text_set = []
with open('new_dataset.json') as f:
    for line in f:
       input_text_set.append(' '.join(line.split()[1:]))
input_text_set = input_text_set[1:-1]

result_list = []

for idx, input_text in enumerate(input_text_set):
    if idx %50 == 0:
        print(f"prossesd files so long {idx}")
        if idx==300:
            break
    result_output = triplet_llama(input_text)

    try:
        result_dict = json.loads(result_output)
    except json.JSONDecodeError as e:
        print(f"JSON decoding failed at index {idx}: {e}")
        continue

    result_list.append(result_dict)

with open('./output/llama_triplets.json', 'w') as result_file:
    json.dump(result_list, result_file, indent=4)

prossesd files so long 0
prossesd files so long 50
prossesd files so long 100
prossesd files so long 150
prossesd files so long 200
prossesd files so long 250
prossesd files so long 300


In [4]:
from triplet_models.mixtral import triplet_mixtral
import json
import re

with open('new_dataset.json', 'r') as file:
    data = json.load(file)

mixtral_triplets = []
for k, v in data.items():
    if len(mixtral_triplets) > 300:
        break
    result = triplet_mixtral(v)
    json_match = re.search(r'\{.*\}', result, re.DOTALL)
    if json_match:
        json_data = json_match.group(0)
        try:
            result_dict = json.loads(json_data)
            mixtral_triplets.append(result_dict)
        except json.JSONDecodeError as e:
            print(f"JSON decoding failed for key {k}: {e}")
            print("Skipping entry.")
    else:
        print(f"No valid JSON found for key {k}, skipping.")

No valid JSON found for key 37, skipping.
No valid JSON found for key 188, skipping.
No valid JSON found for key 214, skipping.
No valid JSON found for key 245, skipping.
No valid JSON found for key 264, skipping.


In [5]:
with open('./output/mixtral_triplets.json', 'w') as file:
    file.write(json.dumps(mixtral_triplets, indent=4))